In [1]:
# 패키지 import
import random
import math
import time
import pandas as pd
import numpy as np

import torch
import torch.utils.data as data
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
import torch.optim as optim

# 초기설정
# Setup seeds
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

# 학습 및 검증 구현

In [2]:
from utils.dataloader import make_datapath_list, DataTransform, VOCDataset

rootpath = "./data/VOCdevkit/VOC2012/"
train_img_list, train_anno_list, val_img_list, val_anno_list = make_datapath_list(rootpath=rootpath)

color_mean = (0.485, 0.456, 0.406)
color_std = (0.229, 0.224, 0.225)

train_dataset = VOCDataset(train_img_list, train_anno_list, phase="train", transform=DataTransform(input_size=475, color_mean=color_mean, color_std=color_std))

val_dataset = VOCDataset(val_img_list, val_anno_list, phase="val", transform=DataTransform(input_size=475, color_mean=color_mean, color_std=color_std))

batch_size = 8

train_dataloader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

dataloaders_dict ={"train":train_dataloader, "val":val_dataloader}

In [3]:
from utils.pspnet import PSPNet

# ADE20K 모델로드 
net = PSPNet(n_classes=150)

state_dict = torch.load("./weights/pspnet50_ADE20K.pth")
net.load_state_dict(state_dict)

# 분류용 합성곱 층을 21로 바꿈
n_classes = 21
net.decode_feature.classification = nn.Conv2d(in_channels=512, out_channels=n_classes, kernel_size=1, stride=1, padding=0)
net.aux.classification = nn.Conv2d(in_channels=256,out_channels=n_classes, kernel_size=1, stride=1, padding=0)

def weights_init(m):
    if isinstance(m, nn.Conv2d):
        nn.init.xavier_normal_(m.weight.data)
        if m.bias is not None:
            nn.init.constant_(m.bias, 0.0)

net.decode_feature.classification.apply(weights_init)
net.aux.classification.apply(weights_init)

print("네트워크 설정 완료: 학습된 가중치를 로드했습니다")


네트워크 설정 완료: 학습된 가중치를 로드했습니다


In [4]:
# 손실 함수 정의
class PSPLoss(nn.Module):
    def __init__(self, aux_weight=0.4):
        super(PSPLoss, self).__init__()
        self.aux_weight = aux_weight

    def forward(self, outputs, targets):

        loss = F.cross_entropy(outputs[0], targets, reduction="mean")
        loss_aux = F.cross_entropy(outputs[1], targets, reduction="mean")

        return loss + self.aux_weight * loss_aux

criterion = PSPLoss(aux_weight=0.4)

# 스케줄러로 에폭별 학습 비율 변경
- 에폭에 따라 학습률을 변화시키는 스케줄러 사용

In [5]:
optimizer = optim.SGD([
    {"params": net.feature_conv.parameters(), "lr":1e-3},
    {"params": net.feature_res_1.parameters(), "lr":1e-3},
    {"params": net.feature_res_2.parameters(), "lr":1e-3},
    {"params": net.feature_dilated_res_1.parameters(), "lr":1e-3},
    {"params": net.feature_dilated_res_2.parameters(), "lr":1e-3},
    {"params": net.pyramid_pooling.parameters(), "lr":1e-3},
    {"params": net.decode_feature.parameters(), "lr":1e-2},
    {"params": net.aux.parameters(), "lr":1e-2},
], momentum=0.9, weight_decay=0.0001)

def lambda_epoch(epoch):
    max_epoch = 30
    return math.pow((1-epoch / max_epoch), 0.9)

scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda_epoch)

In [6]:
def train_model(net, dataloaders_dict, criterion, scheduler, optimizer, num_epochs):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("사용 장치: ", device)

    net.to(device)
    torch.backends.cudnn.benchmark = True

    num_train_imgs = len(dataloaders_dict["train"].dataset)
    num_val_imgs = len(dataloaders_dict["val"].dataset)
    batch_size = dataloaders_dict["train"].batch_size

    iteration = 1
    logs = []

    batch_multiplier = 3

    for epoch in range(num_epochs):
        t_epoch_start = time.time()
        t_iter_start = time.time()

        epoch_train_loss = 0.0
        epoch_val_loss = 0.0

        print("-----------------------")
        print("Epoch {}/{}".format(epoch+1, num_epochs))
        print("-----------------------")

        for phase in ["train", "val"]:
            if phase == "train":
                net.train()
                scheduler.step()
                optimizer.zero_grad()
                print(" (train) ")

            else:
                if((epoch+1) % 5 == 0):
                    net.eval()
                    print("-----------------------")
                    print(" (val) ")
                else:
                    continue

            count = 0
            for imges, anno_class_imges in dataloaders_dict[phase]:
                if imges.size()[0] == 1:
                    continue

                imges = imges.to(device)
                anno_class_imges = anno_class_imges.to(device)

                if (phase == "train") and (count == 0):
                    optimizer.step()
                    optimizer.zero_grad()
                    count = batch_multiplier

                with torch.set_grad_enabled(phase =="train"):
                    outputs = net(imges)
                    loss = criterion(outputs, anno_class_imges.long()) / batch_multiplier

                    if phase == "train":
                        loss.backward()
                        count -= 1

                        if (iteration % 10 == 0):
                            t_iter_finish = time.time()
                            duration = t_iter_finish - t_iter_start
                            print("반복 {} || Loss: {:4f} || 10iter: {:.4f} sec.".format(iteration, loss.item()/batch_size*batch_multiplier, duration))
                            t_iter_start = time.time()

                        epoch_train_loss += loss.item() * batch_multiplier
                        iteration += 1

                    else:
                        epoch_val_loss += loss.item() * batch_multiplier
        
        t_epoch_finish = time.time()
        print("-------------")
        print('epoch {} || Epoch_TRAIN_Loss:{:.4f} ||Epoch_VAL_Loss:{:.4f}'.format(
            epoch+1, epoch_train_loss/num_train_imgs, epoch_val_loss/num_val_imgs))
        print('timer:  {:.4f} sec.'.format(t_epoch_finish - t_epoch_start))
        t_epoch_start = time.time()

        log_epoch = {"epoch" : epoch+1, "train_loss":epoch_train_loss / num_train_imgs, "val_loss":epoch_val_loss/num_val_imgs}
        logs.append(log_epoch)

        df = pd.DataFrame(logs)
        df.to_dsv("log_output.csv")
    
    torch.save(net.state_dict(), "weights/pspnet50_"+str(epoch+1)+".pth")

    



In [7]:
num_epochs = 30
train_model(net, dataloaders_dict, criterion, scheduler, optimizer, num_epochs)

사용 장치:  cuda:0


RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.